In [1]:
# This section enables to use the module code referenced in the repo
import os
import sys

In [15]:
# Imports for the excercise
import pandas as pd 
import logging
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from azureml.core import Workspace, Dataset, Datastore, Experiment, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core import ScriptRunConfig, RunConfiguration, Experiment
from azureml.widgets import RunDetails

## Connecting to a remote AzureML environment

In [7]:
work_env =  Workspace.from_config()

## Starting an experiment with runs and logging to it

In [8]:
exp = Experiment(workspace=work_env, name='Repo')

## Local experiment

### Training script file

In [12]:
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails


def get_training_script():
    # Copyright (c) Microsoft Corporation. All rights reserved.
    # Licensed under the MIT License.

    import argparse

    print("*********************************************************")
    print("Hello Azure ML!")

    parser = argparse.ArgumentParser()
    parser.add_argument('--numbers-in-sequence', type=int, dest='num_in_sequence', default=10,
                        help='number of fibonacci numbers in sequence')

    # This is how you can use a bool argument in Python. If you want the 'my_bool_var' to be True, just pass it
    # in Estimator's script_param as script+params:{'my_bool_var': ''}.
    # And, if you want to use it as False, then do not pass it in the Estimator's script_params.
    # You can reverse the behavior by setting action='store_false' in the next line.
    parser.add_argument("--my_bool_var", action='store_true')

    args = parser.parse_args()
    num = args.num_in_sequence
    my_bool_var = args.my_bool_var


    def fibo(n):
        if n < 2:
            return n
        else:
            return fibo(n - 1) + fibo(n - 2)


    try:
        from azureml.core import Run
        run = Run.get_context()
        print("The value of boolean parameter 'my_bool_var' is {}".format(my_bool_var))
        print("Log Fibonacci numbers.")
        for i in range(0, num - 1):
            run.log('Fibonacci numbers', fibo(i))
        run.complete()
    except:
        print("Warning: you need to install Azure ML SDK in order to log metrics.")

    print("*********************************************************")

In the next cell we take the above function and save the body of the function in the script_file.py

In [13]:
import inspect
script_file_name = 'script_file.py'
lines = inspect.getsource(get_training_script)
lines = '\n'.join(lines.split('\n')[1:])
with open(script_file_name, "w") as script_file:
    script_file.write(lines)

In [24]:
%%writefile script_file.py

import argparse

print("*********************************************************")
print("Hello Azure ML!")

parser = argparse.ArgumentParser()
parser.add_argument('--numbers-in-sequence', type=int, dest='num_in_sequence', default=10,
                    help='number of fibonacci numbers in sequence')

# This is how you can use a bool argument in Python. If you want the 'my_bool_var' to be True, just pass it
# in Estimator's script_param as script+params:{'my_bool_var': ''}.
# And, if you want to use it as False, then do not pass it in the Estimator's script_params.
# You can reverse the behavior by setting action='store_false' in the next line.
parser.add_argument("--my_bool_var", action='store_true')

args = parser.parse_args()
num = args.num_in_sequence
my_bool_var = args.my_bool_var

print(args)

def fibo(n):
    if n < 2:
        return n
    else:
        return fibo(n - 1) + fibo(n - 2)


try:
    from azureml.core import Run
    run = Run.get_context()
    print("The value of boolean parameter 'my_bool_var' is {}".format(my_bool_var))
    print("Log Fibonacci numbers.")
    for i in range(0, num - 1):
        run.log('Fibonacci numbers', fibo(i))
    run.complete()
except:
    print("Warning: you need to install Azure ML SDK in order to log metrics.")

print("*********************************************************")

Overwriting script_file.py


### Local execution of the script

In [25]:
run_local = RunConfiguration()
run_local.environment.python.user_managed_dependencies = True

In [26]:
# run a trial from the train.py code in your current directory
config = ScriptRunConfig(source_directory='.', script=script_file_name, run_config = run_local)
script_run = exp.submit(config)
script_run.wait_for_completion(show_output=True)

RunId: Repo_1589452310_b3a8173c
Web View: https://ml.azure.com/experiments/Repo/runs/Repo_1589452310_b3a8173c?wsid=/subscriptions/c1537527-c126-428d-8f72-1ac9f2c63c1f/resourcegroups/codit-ai-incubators/workspaces/codit-ai-incubators-ml

Streaming azureml-logs/70_driver_log.txt

Entering context manager injector. Current time:2020-05-14T12:31:55.347133
Starting the daemon thread to refresh tokens in background for process with pid = 20637
Preparing to call script [ script_file.py ] with arguments: []
After variable expansion, calling script [ script_file.py ] with arguments: []

*********************************************************
Hello Azure ML!
Namespace(my_bool_var=False, num_in_sequence=10)
*********************************************************


The experiment failed. Finalizing run...
Error occurred when setting run status to Finalizing: 'NoneType' object has no attribute 'get_process_name'
Traceback (most recent call last):
  File "azureml-setup/context_manager_injector.p

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "ServiceError",
        "message": "Failed to run ['python', 'azureml-setup/run_script.py', 'python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'RunHistory:context_managers.RunHistory', '-i', 'TrackUserError:context_managers.TrackUserError', 'script_file.py'] \n Exit code 1 \nDetails can be found in azureml-logs/60_control_log.txt log file.",
        "details": [],
        "messageParameters": {}
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"ServiceError\",\n        \"message\": \"Failed to run ['python', 'azureml-setup/run_script.py', 'python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'RunHistory:context